In [1]:
import pandas as pd
import my_utils as mu

In [2]:
df = mu.get_rawdata('EMERALD')

get summonerId....


100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


get puuId.....


100%|██████████| 12/12 [00:07<00:00,  1.61it/s]


get match_id....


100%|██████████| 12/12 [00:08<00:00,  1.47it/s]


get matches & timeline....


100%|██████████| 36/36 [00:57<00:00,  1.59s/it]


In [3]:
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane[0] == lane[0]) and (tower_team == team):
        return 1
    else:
        return 0
    

In [4]:
def assi_calc(y):
    try:
        return ','.join(list(map(lambda x: str(x), y['assistingParticipantIds'])))
    except:
        return ''

In [5]:
df_creates = []
for i in range(len(df)):
    try:
        match = df.iloc[i].matches['info']['participants']
        for j in range(len(match)):
            match_id = df.iloc[i].match_id
            game_duration = df.iloc[i].matches['info']['gameDuration']
            game_version = df.iloc[i].matches['info']['gameVersion']
            try:
                riotIdGameName = match[j]['riotIdGameName']
            except:
                riotIdGameName = ''
            try:
                riotIdTagline = match[j]['riotIdTagline']
            except:
                riotIdTagline = ''
            summoner_level = match[j]['summonerLevel']
            participants_id = match[j]['participantId']
            champion_name = match[j]['championName']
            champ_experience = match[j]['champExperience']
            team_position = match[j]['teamPosition']
            team_id = match[j]['teamId']
            win = match[j]['win']
            kills = match[j]['kills']
            deaths = match[j]['deaths']
            assists = match[j]['assists']
            total_damage_dealt = match[j]['totalDamageDealtToChampions']
            total_damage_taken = match[j]['totalDamageTaken']


            df_list = list(map(lambda x: x['events'],df.iloc[i].timelines['info']['frames']))
            event_list = [element for array in df_list for element in array]
            tower_log = list(filter(lambda x:x['type'] == 'BUILDING_KILL' ,event_list))

            try:
                ft_tower_lane = tower_log[0]['laneType']
                ft_tower_team = tower_log[0]['teamId']
            except:
                ft_tower_lane = 'n'
                ft_tower_team = 'n'
            ft = lane_processing(ft_tower_lane,ft_tower_team,team_position,team_id)

            tower_tmp = list(map(lambda x: (x['laneType'],x['teamId'],x['timestamp']) ,tower_log))

            try:
                laneTower = list(filter(lambda x:(x[0][0] == team_position[0]) & (x[1] == team_id) ,tower_tmp))[0]
                lane_flag = 1
                laneTowerTime = laneTower[-1]
            except:
                lane_flag = 0
                laneTowerTime = 0
            
            blue_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][0]['bans']))
            red_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][1]['bans']))
            ban_list = '|'.join(list(set(blue_ban + red_ban)))
            
            
            kill_log = list(map(lambda x: list(filter(lambda z: z['type']=='CHAMPION_KILL',x['events'])),df.iloc[i].timelines['info']['frames']))
            kill_log = [element for array in kill_log for element in array]
            k = '|'.join(list(map(lambda x : str(x['killerId']) , kill_log)))
            v = '|'.join(list(map(lambda x :str(x['victimId']), kill_log)))
            a = '|'.join(list(map(lambda x : assi_calc(x) ,kill_log)))
            
            try:
                g15 = df.iloc[i].timelines['info']['frames'][15]['participantFrames'][(str(j+1))]['totalGold']
            except:
                g15 = 0
        
            df_creates.append([match_id,game_duration,game_version,riotIdGameName,riotIdTagline,summoner_level,participants_id,
                              champion_name,champ_experience,team_position,team_id,win,kills,deaths,assists,total_damage_dealt,total_damage_taken,
                              ft,lane_flag,laneTowerTime,ban_list,k,v,a,g15])
    except:
        continue
columns =['matchId', 'gameDuration', 'gameVersion', 'riotIdGameName','riotIdTagline', 
           'summonerLevel', 'participantId', 'championName', 'champExperience', 
           'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 'totalDamageTaken',
         'firstDT','laneTower','LaneTowerTime','bans','killerId','victimId','assistId','G15']
df2 = pd.DataFrame(df_creates , columns = columns)

In [6]:
df2

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,totalDamageDealtToChampions,totalDamageTaken,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15
0,KR_6799052421,2524,13.22.541.9804,,,378,1,Jayce,24955,TOP,...,27014,30264,0,1,959925,517|24|518|555|235|39|238|104|122|53,6|7|9|2|7|9|8|3|10|9|7|7|3|7|8|9|9|9|7|7|3|9|6...,1|4|5|9|2|5|2|7|5|4|5|2|7|1|3|2|4|5|1|2|7|4|1|...,"7|9,10|7,10|4,5|9,10|10|7|2|8,9|7,10|9,10||1,2...",4569
1,KR_6799052421,2524,13.22.541.9804,,,238,2,LeeSin,16978,JUNGLE,...,19110,67016,0,0,0,517|24|518|555|235|39|238|104|122|53,6|7|9|2|7|9|8|3|10|9|7|7|3|7|8|9|9|9|7|7|3|9|6...,1|4|5|9|2|5|2|7|5|4|5|2|7|1|3|2|4|5|1|2|7|4|1|...,"7|9,10|7,10|4,5|9,10|10|7|2|8,9|7,10|9,10||1,2...",5189
2,KR_6799052421,2524,13.22.541.9804,,,74,3,Xerath,27884,MIDDLE,...,74212,18971,0,1,1186730,517|24|518|555|235|39|238|104|122|53,6|7|9|2|7|9|8|3|10|9|7|7|3|7|8|9|9|9|7|7|3|9|6...,1|4|5|9|2|5|2|7|5|4|5|2|7|1|3|2|4|5|1|2|7|4|1|...,"7|9,10|7,10|4,5|9,10|10|7|2|8,9|7,10|9,10||1,2...",6519
3,KR_6799052421,2524,13.22.541.9804,,,381,4,Kalista,18988,BOTTOM,...,22125,33984,1,1,863640,517|24|518|555|235|39|238|104|122|53,6|7|9|2|7|9|8|3|10|9|7|7|3|7|8|9|9|9|7|7|3|9|6...,1|4|5|9|2|5|2|7|5|4|5|2|7|1|3|2|4|5|1|2|7|4|1|...,"7|9,10|7,10|4,5|9,10|10|7|2|8,9|7,10|9,10||1,2...",4275
4,KR_6799052421,2524,13.22.541.9804,,,205,5,Karma,15537,UTILITY,...,13339,27088,0,0,0,517|24|518|555|235|39|238|104|122|53,6|7|9|2|7|9|8|3|10|9|7|7|3|7|8|9|9|9|7|7|3|9|6...,1|4|5|9|2|5|2|7|5|4|5|2|7|1|3|2|4|5|1|2|7|4|1|...,"7|9,10|7,10|4,5|9,10|10|7|2|8,9|7,10|9,10||1,2...",3211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,KR_6799456248,2498,13.22.541.9804,,,227,6,Camille,22487,TOP,...,31484,44545,0,1,1217874,24|-1|233|897|142|887|360|54|21|53,4|10|8|9|4|7|2|8|6|7|4|2|4|6|3|3|8|2|4|8|2|4|9...,10|5|3|5|9|4|10|3|1|2|10|9|10|2|7|6|3|8|10|3|6...,"5|9|7|||9,10|5||7|6|5|5|5|7|1,2|1||3|5|10|1,5|...",5763
286,KR_6799456248,2498,13.22.541.9804,,,283,7,LeeSin,16858,JUNGLE,...,16983,54919,0,0,0,24|-1|233|897|142|887|360|54|21|53,4|10|8|9|4|7|2|8|6|7|4|2|4|6|3|3|8|2|4|8|2|4|9...,10|5|3|5|9|4|10|3|1|2|10|9|10|2|7|6|3|8|10|3|6...,"5|9|7|||9,10|5||7|6|5|5|5|7|1,2|1||3|5|10|1,5|...",5476
287,KR_6799456248,2498,13.22.541.9804,,,121,8,Zed,19930,MIDDLE,...,36434,38951,0,1,1353439,24|-1|233|897|142|887|360|54|21|53,4|10|8|9|4|7|2|8|6|7|4|2|4|6|3|3|8|2|4|8|2|4|9...,10|5|3|5|9|4|10|3|1|2|10|9|10|2|7|6|3|8|10|3|6...,"5|9|7|||9,10|5||7|6|5|5|5|7|1,2|1||3|5|10|1,5|...",6479
288,KR_6799456248,2498,13.22.541.9804,,,362,9,Caitlyn,19829,BOTTOM,...,31389,27659,0,1,957558,24|-1|233|897|142|887|360|54|21|53,4|10|8|9|4|7|2|8|6|7|4|2|4|6|3|3|8|2|4|8|2|4|9...,10|5|3|5|9|4|10|3|1|2|10|9|10|2|7|6|3|8|10|3|6...,"5|9|7|||9,10|5||7|6|5|5|5|7|1,2|1||3|5|10|1,5|...",6874


In [7]:
blue_team = df2[df2.teamId == 100]
red_team = df2[df2.teamId == 200]

In [8]:
tmp = red_team[['matchId','gameVersion','teamPosition','championName','G15']].rename(columns = {'championName':'enemy_champ', 'G15':'enemy_G15'})

In [9]:
blue_team_merge = pd.merge(blue_team,tmp,on = ['matchId','gameVersion','teamPosition'])

In [10]:
tmp_blue = blue_team[['matchId','gameVersion','teamPosition','championName','G15']].rename(columns = {'championName':'enemy_champ', 'G15':'enemy_G15'})

In [11]:
red_team_merge = pd.merge(red_team,tmp_blue,on = ['matchId','gameVersion','teamPosition'])

In [12]:
team_df = pd.concat([blue_team_merge,red_team_merge])

In [13]:
team_df['win'] = team_df.apply(lambda x :1 if x.win == True else 0,axis=1)

In [14]:
tmp_df = team_df[['matchId','championName','enemy_champ','teamPosition','win']]

In [15]:
tmp_df

,matchId,championName,enemy_champ,teamPosition,win
0,KR_6799052421,Jayce,Kayle,TOP,0
1,KR_6799052421,LeeSin,Shaco,JUNGLE,0
2,KR_6799052421,Xerath,Taliyah,MIDDLE,0
3,KR_6799052421,Kalista,Varus,BOTTOM,0
4,KR_6799052421,Karma,Alistar,UTILITY,0
...,...,...,...,...,...
140,KR_6799456248,Camille,Renekton,TOP,0
141,KR_6799456248,LeeSin,Nidalee,JUNGLE,0
142,KR_6799456248,Zed,Fizz,MIDDLE,0
143,KR_6799456248,Caitlyn,Ashe,BOTTOM,0


In [16]:
tmp_df[['championName','enemy_champ','teamPosition','win']].groupby(['championName','teamPosition']).count()

enemy_champ  win
championName teamPosition                  
Aatrox       TOP                     8    8
Ahri         MIDDLE                  4    4
Akali        MIDDLE                  2    2
Akshan       MIDDLE                  1    1
             UTILITY                 1    1
...                                ...  ...
Zeri         BOTTOM                  2    2
Ziggs        BOTTOM                  1    1
Zilean       UTILITY                 1    1
Zoe          MIDDLE                  1    1
Zyra         UTILITY                 2    2

[143 rows x 2 columns]

In [17]:
cnt = tmp_df[['championName','enemy_champ','teamPosition','win']].groupby(['championName','enemy_champ','teamPosition']).count()

In [18]:
cnt = cnt.rename(columns = {'win':'cnt'})

In [19]:
cnt

cnt
championName enemy_champ teamPosition     
Aatrox       Fiora       TOP             1
             Illaoi      TOP             1
             Irelia      TOP             1
             Kalista     TOP             1
             Renekton    TOP             1
...                                    ...
Ziggs        MissFortune BOTTOM          1
Zilean       Nautilus    UTILITY         1
Zoe          MonkeyKing  MIDDLE          1
Zyra         Karma       UTILITY         1
             Pyke        UTILITY         1

[272 rows x 1 columns]

In [20]:
tmp_df[['championName','teamPosition','win']].groupby(['championName','teamPosition']).sum()

win
championName teamPosition     
Aatrox       TOP             5
Ahri         MIDDLE          0
Akali        MIDDLE          0
Akshan       MIDDLE          1
             UTILITY         1
...                        ...
Zeri         BOTTOM          0
Ziggs        BOTTOM          0
Zilean       UTILITY         1
Zoe          MIDDLE          1
Zyra         UTILITY         1

[143 rows x 1 columns]

In [21]:
cnt = tmp_df[['championName','teamPosition','win']].groupby(['championName','teamPosition']).count()

In [22]:
cnt = cnt.rename({'win':'cnt'})

In [23]:
cnt_win = tmp_df[['championName','enemy_champ','teamPosition','win']].groupby(['championName','enemy_champ','teamPosition']).sum()

In [24]:
cnt_win

win
championName enemy_champ teamPosition     
Aatrox       Fiora       TOP             1
             Illaoi      TOP             1
             Irelia      TOP             0
             Kalista     TOP             0
             Renekton    TOP             1
...                                    ...
Ziggs        MissFortune BOTTOM          0
Zilean       Nautilus    UTILITY         1
Zoe          MonkeyKing  MIDDLE          1
Zyra         Karma       UTILITY         0
             Pyke        UTILITY         1

[272 rows x 1 columns]

In [25]:
result = cnt.join(cnt_win, rsuffix='_win')

In [26]:
result.sort_values(by = 'win',ascending = False)

win  win_win
championName teamPosition enemy_champ              
LeeSin       JUNGLE       Taliyah       13        0
                          JarvanIV      13        0
                          Poppy         13        0
                          Sejuani       13        0
                          Shaco         13        0
...                                    ...      ...
Jax          TOP          Yone           1        0
Jayce        MIDDLE       Katarina       1        1
             TOP          Kayle          1        0
Jinx         BOTTOM       Kaisa          1        1
Lillia       TOP          Fiora          1        0

[272 rows x 2 columns]

In [27]:
def get_match_timeline_df(df):
    # df를 한개로 만들기
    df_creater = []
    print('소환사 스텟 생성중.....')
    for i in tqdm(range(len(df))):       
        # matches 관련된 데이터 
        try:
            if df.iloc[i].matches['info']['gameDuration'] > 900:   # 게임시간이 15분이 안넘을경우에는 패스하기
                for j in range(10):
                    tmp = []
                    tmp.append(df.iloc[i].match_id)
                    tmp.append(df.iloc[i].matches['info']['gameDuration'])
                    tmp.append(df.iloc[i].matches['info']['gameVersion'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['summonerName'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['summonerLevel'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['participantId'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['championName'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['champExperience'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['teamPosition'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['teamId'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['win'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['kills'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['deaths'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['assists'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['totalDamageDealtToChampions'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['totalDamageTaken'])
            #timeline 관련된 데이터
                    for k in range(5,26):
                        try:
                            tmp.append(df.iloc[i].timeline['info']['frames'][k]['participantFrames'][str(j+1)]['totalGold'])
                        except:
                            tmp.append(0)
                    df_creater.append(tmp)
        except:
            print(i)
            continue
    columns = ['gameId','gameDuration','gameVersion','summonerName','summonerLevel','participantId','championName','champExperience',
    'teamPosition','teamId','win','kills','deaths','assists','totalDamageDealtToChampions','totalDamageTaken','g_5','g_6','g_7','g_8','g_9','g_10','g_11','g_12','g_13','g_14','g_15','g_16','g_17',
    'g_18','g_19','g_20','g_21','g_22','g_23','g_24','g_25']
    df = pd.DataFrame(df_creater,columns = columns).drop_duplicates()
    print('df 제작이 완료되었습니다. 현재 df의 수는 %d 입니다'%len(df))
    return df

In [28]:
query_mysql = """
create table lol_datas(gameId varchar(30), gameDuration int, gameVersion varchar(40), summonerName varchar(50),
summonerLevel int, participantId int, championName varchar(50), champExperience int,
teamPosition varchar(50), teamId int, win varchar(50), kills int, deaths int, assists int,
totalDamageDealtToChampions int ,totalDamageTaken int, g_5 int,g_6 int,g_7 int,
g_8 int,g_9 int,g_10 int,g_11 int,g_12 int,g_13 int,g_14 int,
g_15 int,g_16 int,g_17 int,g_18 int,g_19 int,g_20 int,g_21 int,
g_22 int,g_23 int, g_24 int,g_25 int
)
"""

In [29]:
conn = mu.connect_mysql()

In [30]:
mu.sql_execute(conn,query_mysql)

OperationalError: (1050, "Table 'lol_datas' already exists")

In [31]:
conn.commit()

In [ ]:
conn.close()

In [32]:
query_mysql2 = 'ALTER TABLE lol_datas add primary key (gameId,participantId)'

In [33]:
mu.sql_execute(conn,query_mysql2)

OperationalError: (1068, 'Multiple primary key defined')

In [35]:
conn.commit()

In [36]:
def insert_matches_timeline_mysql(row,conn):
    # lambda를 이용해서 progress_apply를 통해 insert할 구문 만들기
    query = (
             f'insert into lol_datas(gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId,'
             f'championName, champExperience, teamPosition, teamId, win, kills, deaths, assists,'
             f'totalDamageDealtToChampions, totalDamageTaken, g_5, g_6, g_7, g_8, g_9, g_10, g_11, g_12 ,g_13,g_14,'
             f'g_15, g_16, g_17, g_18, g_19, g_20, g_21, g_22, g_23, g_24, g_25)'
             f'values(\'{row.gameId}\',{row.gameDuration}, \'{row.gameVersion}\', \'{row.summonerName}\','
             f'{row.summonerLevel}, {row.participantId},\'{row.championName}\',{row.champExperience},'
             f'\'{row.teamPosition}\', {row.teamId}, \'{row.win}\', {row. kills}, {row.deaths}, {row.assists},'
             f'{row.totalDamageDealtToChampions},{row.totalDamageTaken},{row.g_5},{row.g_6},{row.g_7},{row.g_8},'
             f'{row.g_9},{row.g_10},{row.g_11},{row.g_12},{row.g_13},{row.g_14},{row.g_15},{row.g_16},{row.g_17},'
             f'{row.g_18},{row.g_19},{row.g_20},{row.g_21},{row.g_22},{row.g_23},{row.g_24},{row.g_25})'
             f'ON DUPLICATE KEY UPDATE '
             f'gameId = \'{row.gameId}\', gameDuration = {row.gameDuration}, gameVersion = \'{row.gameVersion}\', summonerName= \'{row.summonerName}\','
             f'summonerLevel = {row.summonerLevel},participantId = {row.participantId},championName = \'{row.championName}\','
             f'champExperience = {row.champExperience}, teamPosition = \'{row.teamPosition}\', teamId = {row.teamId},win = \'{row.win}\','
             f'kills = {row. kills}, deaths = {row.deaths}, assists = {row.assists}, totalDamageDealtToChampions = {row.totalDamageDealtToChampions},'
             f'totalDamageTaken = {row.totalDamageTaken},g_5 = {row.g_5},g_6 = {row.g_6},g_7 = {row.g_7},g_8 = {row.g_8},g_9 = {row.g_9},'
             f'g_10 = {row.g_10},g_11 = {row.g_11},g_12 = {row.g_12},g_13 = {row.g_13},g_14 = {row.g_14},g_15 = {row.g_15},g_16 = {row.g_16},g_17 = {row.g_17},'
             f'g_18 = {row.g_18},g_19 = {row.g_19},g_20 = {row.g_20},g_21 = {row.g_21},g_22 = {row.g_22},g_23 = {row.g_23},g_24 = {row.g_24},g_25 = {row.g_25}'
            )
    mysql_execute(query,conn)
    return query


In [37]:
import my_utils as mu
import pandas as pd

In [123]:
conn = mu.connect_mysql()
query = 'select * from lol_datas'
result = mu.sql_execute_dict(conn,query)
conn.close()

In [124]:
result_df = pd.DataFrame(result)

In [125]:
result_df = result_df[(result_df.gameDuration > 900)]

In [126]:
sample = result_df[['gameId','championName','win','teamPosition','g_15','teamId']]

In [127]:
sample

,gameId,championName,win,teamPosition,g_15,teamId
0,KR_6606773837,Shyvana,True,TOP,5150,100
1,KR_6606773837,Gragas,True,UTILITY,4575,100
2,KR_6606773837,Yone,True,MIDDLE,5336,100
3,KR_6606773837,Varus,True,BOTTOM,9438,100
4,KR_6606773837,Sylas,True,JUNGLE,6310,100
...,...,...,...,...,...,...
9671,KR_6871321506,Yone,False,,15400,200
9672,KR_6871321506,Singed,True,,18200,100
9673,KR_6871321506,Yuumi,True,,18200,100
9674,KR_6871321506,Katarina,True,,16125,100


- 챔피언 1vs1 라인전 승률 : G15의 데이터를 바탕으로 승률을 구하기 
- 챔피언 1vs1 게임 승률 : G15가 아니라 WIN의 데이터를 바탕으로 승률을 구하기

In [129]:
conn = mu.connect_mysql()
query = 'select * from lol_datas'
result = mu.sql_execute_dict(conn,query)
conn.close()

In [130]:
result_df = pd.DataFrame(result)

In [131]:
result_df = result_df[(result_df.gameDuration > 900)]

In [132]:
sample

,gameId,championName,win,teamPosition,g_15,teamId
0,KR_6606773837,Shyvana,True,TOP,5150,100
1,KR_6606773837,Gragas,True,UTILITY,4575,100
2,KR_6606773837,Yone,True,MIDDLE,5336,100
3,KR_6606773837,Varus,True,BOTTOM,9438,100
4,KR_6606773837,Sylas,True,JUNGLE,6310,100
...,...,...,...,...,...,...
9671,KR_6871321506,Yone,False,,15400,200
9672,KR_6871321506,Singed,True,,18200,100
9673,KR_6871321506,Yuumi,True,,18200,100
9674,KR_6871321506,Katarina,True,,16125,100


In [133]:
blue_team = sample[sample.teamId == 100]
red_team = sample[sample.teamId==200]

In [134]:
blue_tmp = blue_team[['gameId','championName','teamPosition','g_15']]
red_tmp = red_team[['gameId','championName','teamPosition','g_15','win']]

In [135]:
sample_red.columns = 'gameId','enemy_champ','enemy_teamPosition','enemy_g15'
sample_blue.columns = 'gameId','championName','teamPosition','g_15','win'

In [136]:
blue_team = pd.merge(sample_blue,sample_red,how='inner',on=['gameId'])

In [141]:
blue_team

,gameId,championName,teamPosition,g_15,win,enemy_champ,enemy_teamPosition,enemy_g15,lane_win
0,KR_6633453691,Aatrox,TOP,5169,0,Fiora,TOP,5368,0
1,KR_6633453691,Aatrox,TOP,5169,0,Warwick,JUNGLE,5340,0
2,KR_6633453691,Aatrox,TOP,5169,0,Zed,MIDDLE,6171,0
3,KR_6633453691,Aatrox,TOP,5169,0,Samira,BOTTOM,6825,0
4,KR_6633453691,Aatrox,TOP,5169,0,Nautilus,UTILITY,3900,1
...,...,...,...,...,...,...,...,...,...
966,KR_6871218963,Pyke,UTILITY,5179,1,Jax,TOP,4427,1
967,KR_6871218963,Pyke,UTILITY,5179,1,Vi,JUNGLE,6142,0
968,KR_6871218963,Pyke,UTILITY,5179,1,Neeko,MIDDLE,5083,1
969,KR_6871218963,Pyke,UTILITY,5179,1,Samira,BOTTOM,7345,0


In [138]:
blue_team['win'] = blue_team.apply(lambda x :1 if x['win']=='True' else 0,axis=1)

In [140]:
blue_team['lane_win'] = blue_team.apply(lambda x: 1 if x['g_15']>x['enemy_g15'] else 0, axis=1)

In [155]:
games = blue_team[['championName','teamPosition','enemy_champ','win']].groupby(['championName','teamPosition','enemy_champ']).count().rename(columns={'win':'cnt'})

In [156]:
game_win = blue_team[['championName','teamPosition','enemy_champ','win']].groupby(['championName','teamPosition','enemy_champ']).sum()

In [157]:
lane_win = blue_team[['championName','teamPosition','enemy_champ','lane_win']].groupby(['championName','teamPosition','enemy_champ']).sum().rename(columns={'win':'lane_win'})

In [161]:
result_df = lane_win.join(game_win).join(games)

In [164]:
result_df.sort_values(by = 'cnt',ascending=False)[:20]

lane_win  win  cnt
championName teamPosition enemy_champ                    
Ezreal       BOTTOM       Kaisa               2    2    4
                          Renekton            1    1    3
Lucian       BOTTOM       Ezreal              1    2    3
Kaisa        BOTTOM       Yasuo               1    3    3
Yone         TOP          Aatrox              0    1    3
Darius       TOP          Vayne               0    1    2
Ashe         UTILITY      Lux                 0    0    2
Yone         TOP          Xerath              1    2    2
Lucian       BOTTOM       Ashe                0    0    2
Zed          MIDDLE       Khazix              1    1    2
LeeSin       JUNGLE       Jax                 1    1    2
Lucian       BOTTOM       Orianna             2    1    2
Milio        UTILITY      Aphelios            0    0    2
Orianna      MIDDLE       Kaisa               1    2    2
LeeSin       JUNGLE       Pantheon            0    1    2
Akshan       MIDDLE       Ezreal              2    2    2
Ezreal       BOTTOM       Aatrox              2    0    2
Xerath       UTILITY      Kaisa               0    2    2
Milio        UTILITY      Shaco               0    0    2
Zed          MIDDLE       Yasuo               1    2    2

In [159]:
lane_win

lane_win
championName teamPosition enemy_champ          
Aatrox                    Gwen                1
                          Illaoi              1
                          Malzahar            1
                          Vladimir            1
             TOP          Ashe                1
...                                         ...
Ziggs        BOTTOM       JarvanIV            0
                          Samira              0
                          Shaco               1
                          Sylas               0
                          Xerath              0

[929 rows x 1 columns]

In [43]:
result_df.sort_values('games',ascending = False)

KeyError: 'games'

In [ ]:
result_df.sort_values('games',ascending = False)